In [14]:
from bs4 import BeautifulSoup
from IPython.display import Image, display

In [2]:
file = "/kaggle/input/recetas/receta2.html"

#Cargamos el archivo html
with open(file, "r", encoding='utf-8') as file:
    html_content = file.read()

In [3]:
soup = BeautifulSoup(html_content, "html.parser")

In [4]:
#Extraemos el título
title = soup.find("meta",{"property": "og:title"}) ["content"]
title

'Rotisserie Chicken'

In [5]:
# Valoracion de la Receta
valoracion = soup.find("div", class_="mm-recipes-review-bar__rating")
print ("Valoracion: ", valoracion.text)

Valoracion:  4.7


In [6]:
#time_section = soup.find_all("div", class_="mm-recipes-details__label")
#time_section
#for time_section in time_section:
#    print(time_section.text.strip())
# Buscar todos los elementos <div> con clase 'mm-recipes-details__label'
time_labels = soup.find_all("div", class_="mm-recipes-details__label")

# Extraer el tiempo total usando una list comprehension
total_time = [
    label.find_next_sibling("div", class_="mm-recipes-details__value").get_text(strip=True)
    for label in time_labels
    if label.get_text(strip=True) == "Total Time:"
][0]  # Obtener el primer (y único) resultado

print("Tiempo total:", total_time)

Tiempo total: 1 hr 30 mins


In [8]:
# Extracting the description
description = soup.find("meta", {"name": "description"})["content"]
description

"Rotisserie chicken that's easy to cook on a gas grill and turns out moist and juicy with crispy skin. This is a simple recipe that our family loves."

In [9]:
# Ingredientes
ingredients_section = soup.find_all("li", class_="mm-recipes-structured-ingredients__list-item")
for ingredient in ingredients_section:
    print(ingredient.text.strip())

1 (3 pound) whole chicken
1 pinch salt
¼ cup butter, melted
1 tablespoon salt
1 tablespoon ground paprika
¼ tablespoon ground black pepper


In [11]:
# Extraer Instrucciones
instructions_section = soup.find_all("p", class_="comp mntl-sc-block mntl-sc-block-html")
instructions = [instruction.get_text().strip() for instruction in instructions_section]
instructions

["Intimidated by the idea of making a rotisserie chicken at home? We're here to help. Get your grill and rotisserie attachment ready — you'll want to try this recipe ASAP.",
 "Here's what you'll need to make rotisserie chicken at home:",
 "· Whole Chicken: This recipe is meant for a whole 3-pound chicken. If your chicken is larger or smaller, you'll have to adjust the cooking time.· Butter: Butter keeps the chicken moist and juicy, while giving the seasonings something to stick to.· Seasonings: The rotisserie chicken is simply seasoned with salt, pepper, and paprika.",
 "You'll find the full, step-by-step recipe below — but here's a brief overview of what you can expect when you make this rotisserie chicken:",
 'Season the chicken cavity with salt and place it on a rotisserie (tie the wings and legs if you have twine or string available). Set the grill on high and cook for 10 minutes. Turn the grill down to medium and baste with a mixture of the butter and the seasonings. Close the lid

In [13]:
# Extraer Informacion  Nutricional
nutrition_section = soup.find_all("span", class_="mm-recipes-nutrition-facts-label__nutrient-name mm-recipes-nutrition-facts-label__nutrient-name--has-postfix")
nutrition_facts = [fact.parent.get_text().strip().replace('\n', ' ') for fact in nutrition_section]
nutrition_facts

['Total Fat 25g',
 'Saturated Fat 10g',
 'Cholesterol 117mg',
 'Sodium 1311mg',
 'Total Carbohydrate 1g',
 'Dietary Fiber 1g',
 'Total Sugars 0g',
 'Protein 31g',
 'Vitamin C 1mg',
 'Calcium 22mg',
 'Iron 2mg',
 'Potassium 302mg']

In [20]:
# Buscar el <div> con clase 'img-placeholder'
placeholder_div = soup.find("div", class_="img-placeholder")

# Buscar el <img> dentro de ese div
img_tag = placeholder_div.find("img") if placeholder_div else None

# Obtener la imagen desde 'data-src' o 'data-hi-res-src'
if img_tag:
    image_url = img_tag.get("data-hi-res-src") or img_tag.get("data-src")
    display(Image(url=image_url))
else:
    print("No se encontró la imagen en <div class='img-placeholder'>.")

In [21]:
# Find all the links to other recipes
recipe_links = soup.find_all("a", href=True)

# Filter and print only the links that are likely to be recipes
recipe_urls = []
for link in recipe_links:
    href = link['href']
    if "recipe" in href:
        recipe_urls.append(href)

# Print the recipe URLs
print("Linked Recipes:")
for url in recipe_urls:
    print(url)

Linked Recipes:
https://www.allrecipes.com/authentication/login?regSource=3675&relativeRedirectUrl=%2Frecipe%2F93168%2Frotisserie-chicken%2F
/account/add-recipe
https://www.myrecipes.com/favorites
https://www.allrecipes.com/authentication/logout?relativeRedirectUrl=%2Frecipe%2F93168%2Frotisserie-chicken%2F
https://www.magazines.com/allrecipes-magazine.html?utm_source=allrecipes.com&utm_medium=owned&utm_campaign=i111arr1w2661
https://www.magazines.com/allrecipes-magazine.html
https://www.allrecipes.com/recipes/17562/dinner/
https://www.allrecipes.com/recipes/17057/everyday-cooking/more-meal-ideas/5-ingredients/main-dishes/
https://www.allrecipes.com/recipes/15436/everyday-cooking/one-pot-meals/
https://www.allrecipes.com/recipes/1947/everyday-cooking/quick-and-easy/
https://www.allrecipes.com/recipes/455/everyday-cooking/more-meal-ideas/30-minute-meals/
https://www.allrecipes.com/recipes/17889/everyday-cooking/family-friendly/family-dinners/
https://www.allrecipes.com/recipes/94/soups-s

In [24]:
import requests

def fetch_recipe_content(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,image/apng,*/*;q=0.8",
        "Accept-Language": "en-US,en;q=0.9",
        "Connection": "keep-alive",
        "Upgrade-Insecure-Requests": "1"
    }
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Raise an exception for HTTP errors
        soup = BeautifulSoup(response.content, "html.parser")
        
        # Extract relevant information (title, ingredients, instructions, etc.)
        title = soup.find("meta", {"property": "og:title"})["content"] if soup.find("meta", {"property": "og:title"}) else "No title found"
    

        description = soup.find("meta", {"name": "description"})["content"] if soup.find("meta", {"name": "description"}) else "No description found"
        
        ingredients_section = soup.find_all("span", class_="recipe-ingredients__ingredient")
        ingredients = [ingredient.get_text(strip=True) for ingredient in ingredients_section] if ingredients_section else ["No ingredients found"]

        instructions_section = soup.find_all("li", class_="mntl-sc-block-group--LI mntl-sc-block mntl-sc-block-html")
        instructions = [instruction.get_text(strip=True) for instruction in instructions_section] if instructions_section else ["No instructions found"]
        
        # Print the extracted information for each recipe
        print("Recipe Title:", title)
        print("Description:", description)
        print("Ingredients:")
        for ingredient in ingredients:
            print("-", ingredient)
        print("Instructions:")
        for i, instruction in enumerate(instructions, 1):
            print(f"{i}. {instruction}")
        print("\n")
    except requests.exceptions.RequestException as e:
        print(f"Failed to fetch {url}: {e}")

# Loop through the list of recipe URLs and fetch their contents
for url in recipe_urls[:2]:
    fetch_recipe_content(url)

Recipe Title: No title found
Description: No description found
Ingredients:
- No ingredients found
Instructions:
1. No instructions found


Failed to fetch /account/add-recipe: Invalid URL '/account/add-recipe': No scheme supplied. Perhaps you meant https:///account/add-recipe?
